# *Challenge 1*: A **kernel** methods / **DL** pipeline for the FashionMNIST dataset

Advanced Topics in Machine Learning -- Fall 2023, UniTS

<a target="_blank" href="https://colab.research.google.com/github/ganselmif/adv-ml-units/blob/main/notebooks/AdvML_Challenge_1.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>


In [ ]:
## Import Libraries
import numpy as np
import pandas as pd

from tqdm.auto import trange

import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch.utils.data import DataLoader, SubsetRandomSampler, TensorDataset
import torch.optim as optim

from torchvision import datasets, transforms


import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits import mplot3d
from matplotlib import rcParams

from sklearn.decomposition import PCA, KernelPCA

from sklearn.metrics import adjusted_rand_score, accuracy_score, davies_bouldin_score
from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC

# Used to save data into files
import pickle as pkl
import os

# Used to measure time
import time

In [ ]:
## Import train dataset only, scale them and convert to data loader
BATCH_SIZE = 64

train_dataset = datasets.FashionMNIST(
    root="./data",
    train=True,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize(0, 1)]),
    download=True,
)

print(f"train_dataset:\n {train_dataset}")

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE)

We will use the original training dataset (composed of 60000 elements) and keep only 10000 of them. Of these, 7000 will be the training dataset, and 3000 will be the test dataset.

In [ ]:
subset_size = 10000
test_percentage = 0.3

## set a seed for randperm
th.manual_seed(42)
idx = th.randperm(len(train_dataset))[: subset_size]
print(len(idx))
sampler = SubsetRandomSampler(idx)
train_subset_loader = DataLoader(train_dataset, sampler=sampler)

test_subset_loader = DataLoader(train_dataset, sampler=sampler)

In [ ]:
## Convert the images and their labels to numpy arrays and reshape them to vectors
labels = np.array([])
images = np.array([])
for batch in train_subset_loader:
    data, label = batch
    images = np.append(images, data.numpy().reshape(1, -1))
    labels = np.append(labels, label.numpy())

train_subset_scaled = images.reshape(subset_size, -1)
labels = labels.astype(int)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    train_subset_scaled, labels, test_size=0.3, random_state=42, shuffle=False
)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# Creating dictionary of labels for better understanding
description = {0: "T-shirt/top", 
               1: "Trouser", 
               2: "Pullover", 
               3: "Dress", 
               4: "Coat", 
               5: "Sandal", 
               6: "Shirt", 
               7: "Sneaker", 
               8: "Bag", 
               9: "Ankle boot"}

ticks = list(description.keys())
tick_labels = list(description.values())

In [ ]:
## Defining functions to save and load data from pickle files

def save_data(data, filename):
    if not os.path.exists(filename):
        with open(filename, "wb") as f:
            pkl.dump(data, f)

def load_data(filename):
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            data = pkl.load(f)
    return data

In [ ]:
# Printing samples and checking labels
indices = np.random.choice(x_train.shape[0], size=30, replace=False)

def print_img_index(indices, img_set, lab_set, class_number):
    counter = np.zeros(10)

    # Counting number of appearance of each label
    for i in range(len(indices)):
        label = lab_set[indices[i]]
        counter[label] += 1

    # Plotting the images with the labels as titles
    fig, axs = plt.subplots(5, 6, figsize=(10, 10))
    axs = axs.flatten()

    for i, idx in enumerate(indices):
        axs[i].imshow(
            img_set[idx].reshape(28, 28), cmap="gray"
        )  # Assuming images are 28x28 pixels
        axs[i].axis("off")  # Turn off axis labels for cleaner visualization
        # string = str(description[labels_subset[idx]])
        axs[i].set_title(description[lab_set[idx].item()])

    fig.suptitle(f"Class {class_number}")

    # Printing the counted amounts
    for j in range(10):
        print(f"{description[j]}: {counter[j]}")


print_img_index(indices, x_train, y_train, 0)

## Exercise 1

In [ ]:
# Choose the color map for the plots
colors_rgb = [
    (33, 240, 182),
    (21, 122, 72),
    (155, 209, 198),
    (16, 85, 138),
    (172, 139, 248),
    (133, 22, 87),
    (197, 81, 220),
    (56, 181, 252),
    (18, 85, 211),
    (171, 230, 91),
]
colors_rgb_normalized = colors_rgb_normalized = np.array(colors_rgb) / 255.0
cmap = ListedColormap(colors_rgb_normalized)
plt.rcParams["ps.useafm"] = True
title_dict = {
    "fontname": "Sans-serif",
    "fontsize": 16,
    "fontweight": "bold",
}
rcParams.update({"figure.autolayout": True})

bar_color = (16, 85, 138)
bar_rgb_color = np.array(bar_color) / 255.0

In [ ]:
## Perform linear PCA

model = PCA(n_components = 3)
data_pca_linear = model.fit_transform(x_train, y_train)

del(model)

In [ ]:
# Plot the first two principal components

fig, ax = plt.subplots(figsize=(9, 6), dpi=200)
p = plt.scatter(
    data_pca_linear[:, 0], data_pca_linear[:, 1], c=y_train, marker=".", cmap=cmap
)
# legend = ax.legend(*p.legend_elements(), loc="right", title="Classes")
# ax.set_xlim(left=None,right=13)
# ax.add_artist(legend)
plt.xlabel("Principal Component 1", fontsize=11)
plt.ylabel("Principal \n Component 2", fontsize=11, rotation=0, labelpad=50)
plt.xticks([])
plt.yticks([])
# plt.title("First 2 components of linear PCA", fontdict=title_dict)
plt.savefig("Report/pca_linear_2comps.png")
##plt.show()

del (p, fig, ax)

In [ ]:
# Plot the first three principal components

fig = plt.figure(figsize=(9, 9), dpi=200)
ax = fig.add_subplot(111, projection="3d")

for i in range(3):
    p = ax.scatter(
        data_pca_linear[:, 0],
        data_pca_linear[:, 1],
        data_pca_linear[:, 2],
        c=y_train,
        marker=".",
        cmap=cmap,
    )
ax.view_init(elev=30, azim=30)

ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.yaxis.set_rotate_label(False)  # disable automatic rotation
ax.xaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_xlabel("Principal \n Component 1", fontsize=11, labelpad=15)
ax.set_ylabel("Principal \n Component 2", fontsize=11, labelpad=15)
ax.set_zlabel("Principal \n Component 3", fontsize=11, labelpad=15)
xticks = ax.get_xticks()
yticks = ax.get_yticks()
zticks = ax.get_zticks()
empty_labels_x = ["" for i in range(len(xticks))]
empty_labels_y = ["" for i in range(len(yticks))]
empty_labels_z = ["" for i in range(len(zticks))]
ax.set_xticks(xticks, empty_labels_x)
ax.set_yticks(yticks, empty_labels_y)
ax.set_zticks(zticks, empty_labels_z) 

# legend = ax.legend(*p.legend_elements(), loc="right", title="Classes")
# ax.add_artist(legend)
ax.dist = 13
###plt.show()

del fig
del ax
del p

plt.savefig("Report/pca_linear_3comps.png")

#### Comment
The data does not seem to be well separated, so finding the right hyperplane for classification will be hard.

### 1.2: Perform kernel PCA

In [ ]:
# Perform kernel pca using the RBF kernel
    
kernel_pca = KernelPCA(kernel="rbf", n_components = 3)
data_pca_rbf = kernel_pca.fit_transform(x_train)

In [ ]:
# # Plot the first 2 principal components

fig = plt.figure(figsize=(9, 6), dpi=200)
p = plt.scatter(
    data_pca_rbf[:, 0], data_pca_rbf[:, 1], c=y_train, marker=".", cmap=cmap
)

plt.xlabel("Principal Component 1", fontsize=11)
plt.ylabel("Principal \n Component 2", fontsize=11, rotation=0, labelpad=50)
plt.xticks([])
plt.yticks([])
plt.savefig("Report/pca_rbf_2comps.png")
##plt.show()

del (p, fig)

In [ ]:
# Plot the first 3 principal components

fig = plt.figure(figsize=(9, 9), dpi=200)
ax = fig.add_subplot(111, projection="3d")

for i in range(3):
    p = ax.scatter(
        data_pca_rbf[:, 0],
        data_pca_rbf[:, 1],
        data_pca_rbf[:, 2],
        c=y_train,
        marker=".",
        cmap=cmap,
    )

ax.view_init(elev=30, azim=30)
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.yaxis.set_rotate_label(False)  # disable automatic rotation
ax.xaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_xlabel("Principal \n Component 1", fontsize=11, labelpad=15)
ax.set_ylabel("Principal \n Component 2", fontsize=11, labelpad=15)
ax.set_zlabel("Principal \n Component 3", fontsize=11, labelpad=15)
xticks = ax.get_xticks()
yticks = ax.get_yticks()
zticks = ax.get_zticks()
empty_labels_x = ["" for i in range(len(xticks))]
empty_labels_y = ["" for i in range(len(yticks))]
empty_labels_z = ["" for i in range(len(zticks))]
ax.set_xticks(xticks, empty_labels_x)
ax.set_yticks(yticks, empty_labels_y)
ax.set_zticks(zticks, empty_labels_z)
ax.dist = 13
###plt.show()

plt.savefig("Report/pca_rbf_3comps.png")

del fig
del ax
del p

### Perform parameter tuning

In [ ]:
kernel_pca = KernelPCA(kernel = "rbf", n_components = 3, gamma = 5 / 784)
data_pca_rbf = kernel_pca.fit_transform(x_train)

In [ ]:
# Perform kernel pca using the RBF kernel, tune gamma to separate clusters
    
gamma = np.array([(1/10)*(1/784), 1/784, 10 * (1/784)])

data_pca_rbf = np.ndarray((int(subset_size*(1-test_percentage)), 3, len(gamma)))

for i in range(len(gamma)):
    kernel_pca = KernelPCA(kernel = "rbf", n_components = 3, gamma = gamma[i])
    data_pca_rbf[:, :, i] = kernel_pca.fit_transform(x_train)

In [ ]:
# scatterplot for different gammas

gammas = ["$\\frac{1}{5} * \\frac{1}{784}$", "$\\frac{1}{784}$", "$5 * \\frac{1}{784}$"]
fig, axs = plt.subplots(1, 3, figsize=(17, 6))

for i, ax in enumerate(axs.flat):
    p = ax.scatter(
        data_pca_rbf[:, 0, i], data_pca_rbf[:, 1, i], c=y_train, marker=".", cmap=cmap
    )
    ax.set_title("Gamma = " + gammas[i])

for ax in axs.flat:
    ax.set_xticks([])
    ax.set_yticks([])


In [ ]:
## Choose the range of the parameter gamma
gammas = np.arange(1/784 - 5 * (1/784) * (1/10), 1/784 + 5 * (1/784) * (1/10), (1/784) * (1/10))

## Extract eigenvalues
n_components = 3
eigenvalues_rbf = np.empty((len(gammas), n_components))

for i in range(len(gammas)):
    kernel_pca = KernelPCA(kernel="rbf", n_components = n_components, gamma = gammas[i])
    eigenvalues_rbf[i] = kernel_pca.fit(x_train).eigenvalues_



fig, axs = plt.subplots(2, 5, figsize=(30, 10))
# Create 10 random plots
for i, ax in enumerate(axs.flat):
    x = np.arange(1, len(eigenvalues_rbf[i, :]) + 1, 1)
    # Plot the data on the corresponding axis
    ax.bar(x, eigenvalues_rbf[i, :], color = bar_rgb_color)
    ax.set_ylim(0, 500)
    ax.set_xticks(range(1, 4))
    # ax.set_xlabel('Component')
    # ax.set_ylabel('Eigenvalue')
    ax.set_title('Gamma = ' + str(np.round(gammas[i], 5)))

### 1.3 Perform kPCA using another kernel

In [ ]:
# Try kernel poly

kernel_pca = KernelPCA(kernel = "poly", n_components = 3)

data_pca_poly = kernel_pca.fit_transform(x_train)

In [ ]:
# Plot the first 2 principal components

fig = plt.figure(figsize=(9, 6), dpi=200)
p = plt.scatter(data_pca_poly[:, 0], data_pca_poly[:, 1], c = y_train, marker='.', cmap = cmap)



plt.xlabel('Principal Component 1', fontsize=11)
plt.ylabel('Principal \n Component 2', fontsize=11, rotation=0, labelpad=50)

plt.savefig("Report/pca_poly_2comps.png")
##plt.show()

del(p, fig)

In [ ]:
# Plot the first 3 principal components

fig = plt.figure(figsize=(9, 9), dpi=200)
ax = fig.add_subplot(111, projection="3d")

for i in range(3):
    p = ax.scatter(
        data_pca_poly[:, 0],
        data_pca_poly[:, 1],
        data_pca_poly[:, 2],
        c=y_train,
        marker=".",
        cmap=cmap,
    )

ax.view_init(elev=30, azim=30)
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.yaxis.set_rotate_label(False)  # disable automatic rotation
ax.xaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_xlabel("Principal \n Component 1", fontsize=11, labelpad=15)
ax.set_ylabel("Principal \n Component 2", fontsize=11, labelpad=15)
ax.set_zlabel("Principal \n Component 3", fontsize=11, labelpad=15)
xticks = ax.get_xticks()
yticks = ax.get_yticks()
zticks = ax.get_zticks()
empty_labels_x = ["" for i in range(len(xticks))]
empty_labels_y = ["" for i in range(len(yticks))]
empty_labels_z = ["" for i in range(len(zticks))]
ax.set_xticks(xticks, empty_labels_x)
ax.set_yticks(yticks, empty_labels_y)
ax.set_zticks(zticks, empty_labels_z)
ax.dist = 13
##plt.show()

plt.savefig("Report/pca_poly_3comps.png")

del (p, fig, ax)

In [ ]:
# Try kernel sigmoid
kernel_pca = KernelPCA(kernel="sigmoid", n_components = 10)

data_pca_sigmoid = kernel_pca.fit_transform(x_train)

eigenvalues_sigmoid = kernel_pca.eigenvalues_

In [ ]:
# Plot the first 2 principal components

fig = plt.figure(figsize=(9, 6), dpi=200)
p = plt.scatter(data_pca_sigmoid[:, 0], data_pca_sigmoid[:, 1], c = y_train, marker='.', cmap = cmap)

# cb = plt.colorbar(p)
# cb.ax.set_title('Class', fontsize=11)
# del(cb)

plt.xlabel('Principal Component 1', fontsize=11)
plt.ylabel('Principal \n Component 2', fontsize=11, rotation=0, labelpad=50)
plt.xticks([])
plt.yticks([])

plt.savefig("Report/pca_sigmoid_2comps.png")
##plt.show()

del(p, fig)

In [ ]:
# Plot the first 3 principal components

fig = plt.figure(figsize=(9, 9), dpi=200)
ax = fig.add_subplot(111, projection="3d")

for i in range(3):
    p = ax.scatter(
        data_pca_sigmoid[:, 0],
        data_pca_sigmoid[:, 1],
        data_pca_sigmoid[:, 2],
        c=y_train,
        marker=".",
        cmap=cmap,
    )

ax.view_init(elev=30, azim=30)
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.yaxis.set_rotate_label(False)  # disable automatic rotation
ax.xaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_xlabel("Principal \n Component 1", fontsize=11, labelpad=15)
ax.set_ylabel("Principal \n Component 2", fontsize=11, labelpad=15)
ax.set_zlabel("Principal \n Component 3", fontsize=11, labelpad=15)
xticks = ax.get_xticks()
yticks = ax.get_yticks()
zticks = ax.get_zticks()
empty_labels_x = ["" for i in range(len(xticks))]
empty_labels_y = ["" for i in range(len(yticks))]
empty_labels_z = ["" for i in range(len(zticks))]
ax.set_xticks(xticks, empty_labels_x)
ax.set_yticks(yticks, empty_labels_y)
ax.set_zticks(zticks, empty_labels_z)

ax.dist = 13
##plt.show()
plt.savefig("Report/pca_sigmoid_3comps.png")

del fig
del ax
del p

In [ ]:
# Measure separation of the clusters using the Davies-Bouldin score
# The lower the better

# DB_score = []

# DB_score.append(davies_bouldin_score(data_pca_linear, y_train.reshape(-1)))
# DB_score.append(davies_bouldin_score(data_pca_rbf, y_train.reshape(-1)))
# DB_score.append(davies_bouldin_score(data_pca_poly, y_train.reshape(-1)))
# DB_score.append(davies_bouldin_score(data_pca_sigmoid, y_train.reshape(-1)))

# print(f"DB score: linear: {DB_score[0]:.4f} | rbf: {DB_score[1]:.4f} | poly: {DB_score[2]:.4f} | sigmoid: {DB_score[3]:.4f}")

## Exercise 2

In [ ]:
# Perform clustering with different techniques

labels_Kmeans = KMeans(n_clusters = 10, n_init=10).fit(data_pca_sigmoid).labels_

labels_Spectral = SpectralClustering(n_clusters = 10, affinity='nearest_neighbors').fit(data_pca_sigmoid).labels_

labels_Gaussian = GaussianMixture(n_components = 10).fit(data_pca_sigmoid).predict(data_pca_sigmoid)

ex2_labels = np.array([y_train, labels_Kmeans, labels_Spectral, labels_Gaussian])

In [ ]:
# Plot the results and compare them with the original clustering

fig, axs = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True, dpi=200)

title_names = ["Original", "K Means", "Spectral Clustering", "Gaussian Mixture"]

for ax, i in zip(axs.flat, range(4)):
    ax.scatter(data_pca_sigmoid[:, 0], data_pca_sigmoid[:, 1], c = ex2_labels[i, :], marker='.', cmap = cmap)
    ax.set_title(title_names[i], fontweight="bold", fontsize=13)
    ax.set_xticks([])
    ax.set_yticks([])

plt.savefig("Report/unsupervised_clustering.png")

In [ ]:
# Calculate Adjusted Rand Index

ARI = np.empty(3)

for i in range(3):
    ARI[i] = adjusted_rand_score(ex2_labels[0, :], ex2_labels[i + 1, :])
    print(f"Adjusted Rand Index for {title_names[i + 1]}: {ARI[i]:.4f}")

In [ ]:
# Measure separation of the clusters using the Davies-Bouldin score
# The lower the better

# DB_score = []

# DB_score.append(davies_bouldin_score(data_pca_sigmoid, labels[0, :].reshape(-1)))
# DB_score.append(davies_bouldin_score(data_pca_sigmoid, labels[1, :].reshape(-1)))
# DB_score.append(davies_bouldin_score(data_pca_sigmoid, labels[2, :].reshape(-1)))
# DB_score.append(davies_bouldin_score(data_pca_sigmoid, labels[3, :].reshape(-1)))

# print(f"DB score: original: {DB_score[0]:.4f} | KMeans: {DB_score[1]:.4f} | Spectral: {DB_score[2]:.4f} | Gaussian: {DB_score[3]:.4f}")

#### a
As we can see, label assignment performed poorly. This, probably, because the clusters are very close to each other and not clearly separated.

#### b
As we can see from the plot below, there is a clear elbow on the third component. This suggests that 10 does not reflect the actual knee point of the spectrum of the principal components.

In [ ]:
# Plot the eigenvalues obtained with the sigmoid method

plt.figure(figsize=(9, 6), dpi=200)
plt.plot(np.arange(1, len(eigenvalues_sigmoid) + 1, 1), eigenvalues_sigmoid)

plt.xticks(np.arange(1, 11, 1))

plt.xlabel('Component')
plt.ylabel('Eigenvalue', rotation=0, labelpad=20)
##plt.show()
# plt.savefig("Report/eigenvalues_sigmoid.png")

In [ ]:
# Unnormalized plot of the eigenvalues
fig = plt.figure(figsize=(9, 6), dpi=200)
plt.bar(
    x=np.arange(1, len(eigenvalues_sigmoid) + 1),
    height=eigenvalues_sigmoid,
    color=bar_rgb_color,
)
xticks = np.arange(1, len(eigenvalues_sigmoid) + 1, 1)
plt.xticks(xticks)
plt.xlabel("Component", fontsize=11)
plt.ylabel("Eigenvalue",  fontsize=11, rotation=0, labelpad=35)
##plt.show()

plt.savefig("Report/eigenvalues_sigmoid.png")

In [ ]:
# Normalized plot of the eigenvalues
fig = plt.figure(figsize=(9, 6), dpi=200)
plt.bar(
    x=np.arange(1, len(eigenvalues_sigmoid) + 1),
    height=eigenvalues_sigmoid / np.max(eigenvalues_sigmoid),
    color=bar_rgb_color,
)
xticks = np.arange(1, len(eigenvalues_sigmoid) + 1, 1)
plt.xticks(xticks)
plt.xlabel("Component", fontsize=11)
plt.ylabel("Eigenvalue", fontsize=11, rotation=0, labelpad=35)
##plt.show()

plt.savefig("Report/eigenvalues_sigmoid_normalized.png")

In [ ]:
del(ARI)
del(eigenvalues_sigmoid)
del(labels_Kmeans)
del(labels_Gaussian)
del(ax)
del(axs)

## Exercise 3

In [ ]:
# Perform sigmoid kernel PCA again to obtain 10000 elements
kernel_pca = KernelPCA(kernel="sigmoid", n_components=3)

data_pca_sigmoid = kernel_pca.fit_transform(train_subset_scaled)

print(f"train_subset_scaled.shape = {train_subset_scaled.shape}")

print(f"data_pca_sigmoid.shape = {data_pca_sigmoid.shape}")

# Now that we have the total 10000 elements of shape 3 again,
# use the trained model to predict 10000 labels and store in labels_Gaussian

labels_Spectral = (
    SpectralClustering(n_clusters=10, affinity="nearest_neighbors")
    .fit(data_pca_sigmoid)
    .labels_
)
print(f"labels_Spectral.shape = {labels_Spectral.shape}")

In [ ]:
def compare_labels(lab1, lab2):
    for i in range(len(lab1)):
        if lab1[i] != lab2[i]:
            return False
    return True


def compare_image_arrays(img_array1, img_array2):
    for i in range(len(img_array1)):
        img1 = img_array1[i]
        img2 = img_array2[i]
        for j in range(len(img1)):
            if not np.array_equal(img1[j], img2[j]):
                return False
    return True

In [ ]:
# Split the dataset into training and test set
# First, get all 10000 (both training and test data) predictions with trained model

x_train_ex3, x_test_ex3, y_train_ex3, y_test_ex3 = train_test_split(
    train_subset_scaled, labels_Spectral, test_size=0.3, random_state=42, shuffle=False
)

In [ ]:
def print_img_index(indices, img_set, lab_set, class_number):
    # print_img_index(indices, x_test_ex3, true_labels, class_number)
    counter = np.zeros(10)

    # Counting number of appearance of each label
    for i in range(len(indices)):
        label = lab_set[indices[i]]
        counter[label] += 1

    # Plotting the images with the labels as titles
    fig, axs = plt.subplots(5, 6, figsize=(10, 10))
    axs = axs.flatten()

    for i, idx in enumerate(indices):
        axs[i].imshow(
            img_set[idx].reshape(28, 28), cmap="gray"
        )  # Assuming images are 28x28 pixels
        axs[i].axis("off")  # Turn off axis labels for cleaner visualization
        # string = str(description[labels_subset[idx]])
        axs[i].set_title(description[lab_set[idx].item()])
        # axs[i].set_title(lab_set[idx].item())

    fig.suptitle(f"Class {class_number}")

    # Printing the counted amounts
    for j in range(10):
        print(f"{description[j]}: {counter[j]}")

In [ ]:
# Gets 30 samples for the specified class according to given labels
def show_class(class_number, learnt_labels, true_labels):
    # show_class(i, labels_CNN, y_test)
    indices = []
    counter = np.zeros(10)
    idx = np.random.choice(len(learnt_labels), size=1, replace=False)[0]

    # Selecting 30 samples with learnt_label = class number
    for i in range(30):
        idx = np.random.choice(len(learnt_labels), size=1, replace=False)[0]
        while learnt_labels[idx] != class_number:
            idx = np.random.choice(len(learnt_labels), size=1, replace=False)[0]

        indices.append(idx)
        counter[true_labels[idx]] += 1

    # Now that the indices are selected, print the images and the corresponding label on which the model was trained
    print_img_index(
        indices=indices,
        img_set=x_test_ex3,
        lab_set=true_labels,
        class_number=class_number,
    )

In [ ]:
for i in range(10):
    print(f"\nClass {i}:")
    show_class(class_number=i, learnt_labels=y_test_ex3, true_labels=y_test)

#### 3.1: kernel SVM with different kernels

In [ ]:
# Linear kernel
classifier = SVC(kernel="linear").fit(x_train_ex3, y_train_ex3)

label_predict_SVC_linear = classifier.predict(x_test_ex3)
acc_linear = accuracy_score(y_test_ex3, label_predict_SVC_linear)

In [ ]:
# RBF kernel

classifier = SVC(kernel="rbf").fit(x_train_ex3, y_train_ex3)

label_predict_SVC_rbf = classifier.predict(x_test_ex3)
acc_rbf = accuracy_score(y_test_ex3, label_predict_SVC_rbf)

In [ ]:
# Polynomial kernel

classifier = SVC(kernel="poly").fit(x_train_ex3, y_train_ex3.reshape(len(y_train_ex3)))

label_predict_SVC_poly = classifier.predict(x_test_ex3)

acc_poly = accuracy_score(y_test_ex3, label_predict_SVC_poly)

In [ ]:
# Sigmoid kernel

classifier = SVC(kernel="sigmoid").fit(
    x_train_ex3, y_train_ex3
)

label_predict_SVC_sigmoid = classifier.predict(x_test_ex3)

acc_sigmoid = accuracy_score(y_test_ex3, label_predict_SVC_sigmoid)

In [ ]:
print(f"Accuracy: linear: {acc_linear:.2f} | rbf: {acc_rbf:.2f} | poly: {acc_poly:.2f} | sigmoid: {acc_sigmoid:.2f}")

In [ ]:
labels_SVC_unsupervised = np.concatenate(
    (
        y_test_ex3.reshape(len(y_test_ex3), 1),
        label_predict_SVC_linear.reshape(len(y_test_ex3), 1),
        label_predict_SVC_rbf.reshape(len(y_test_ex3), 1),
        label_predict_SVC_poly.reshape(len(y_test_ex3), 1),
        label_predict_SVC_sigmoid.reshape(len(y_test_ex3), 1),
    ),
    axis=1,
)

In [ ]:
model = KernelPCA(kernel="sigmoid", n_components=3).fit_transform(x_test_ex3)

In [ ]:
import matplotlib.gridspec as gridspec

titles = ["Original", "Linear", "RBF", "Polynomial", "Sigmoid"]


fig = plt.figure(figsize=(9, 6), dpi=200)
gs = gridspec.GridSpec(2, 6)  # , width_ratios=[1, 1, 1], height_ratios=[1, 1])

# Create subplots
ax1 = plt.subplot(gs[0, :2])
ax2 = plt.subplot(gs[0, 2:4])
ax3 = plt.subplot(gs[0, 4:])
ax4 = plt.subplot(gs[1, 1:3])
ax5 = plt.subplot(gs[1, 3:5])

# Plot data in subplots (replace with your actual plotting code)
ax1.scatter(
    model[:, 0], model[:, 1], c=labels_SVC_unsupervised[:, 0], cmap=cmap, marker="."
)
ax1.title.set_text(titles[0])

ax2.scatter(
    model[:, 0], model[:, 1], c=labels_SVC_unsupervised[:, 1], cmap=cmap, marker="."
)
ax2.title.set_text(titles[1])

ax3.scatter(
    model[:, 0], model[:, 1], c=labels_SVC_unsupervised[:, 2], cmap=cmap, marker="."
)
ax3.title.set_text(titles[2])

ax4.scatter(
    model[:, 0], model[:, 1], c=labels_SVC_unsupervised[:, 3], cmap=cmap, marker="."
)
ax4.title.set_text(titles[3])

ax5.scatter(
    model[:, 0], model[:, 1], c=labels_SVC_unsupervised[:, 4], cmap=cmap, marker="."
)
ax5.title.set_text(titles[4])

for ax in [ax1, ax2, ax3, ax4, ax5]:
    ax.set_xticks([])
    ax.set_yticks([])

# Adjust layout for better spacing
plt.tight_layout()

# Show the plot
##plt.show()

plt.savefig("Report/unsupervised_SVC.png")

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(15, 3), sharex=True, sharey=True)

titles = ["Original", "Linear", "RBF", "Polynomial", "Sigmoid"]

i = 0
for ax in axs:
    ax.scatter(model[:, 0], model[:, 1], c = labels_SVC_unsupervised[:, i], cmap = cmap, marker='.')
    ax.title.set_text(titles[i])
    i += 1

#### 3.2: Fully Connected NN

In [ ]:
## Pass data to tensors
data_train = TensorDataset(
    Tensor(x_train_ex3.reshape(-1, 1, 28, 28)), th.tensor(y_train_ex3, dtype=th.long)
)
data_train_loader = DataLoader(dataset=data_train, batch_size=BATCH_SIZE, shuffle=False)

data_test = TensorDataset(
    Tensor(x_test_ex3.reshape(-1, 1, 28, 28)), th.tensor(y_test_ex3, dtype=th.long)
)
data_test_loader = DataLoader(dataset=data_test, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Decide if you want to train multiple models with different hyperparameters
train_multiple_models = True

In [ ]:
# Define some functions needed to calculate the accuracy

def get_batch_accuracy(logit, target):
    corrects = (th.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects / target.size(0)
    return accuracy.item()


def get_test_stats(model, criterion, test_loader, device):
    test_acc, test_loss = 0.0, 0.0
    for i, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_loss += criterion(outputs, labels).item()
        test_acc += get_batch_accuracy(outputs, labels)
        return test_loss, test_acc

In [ ]:
# Define function used to train the model

def train_model(epochs, train_loader, criterion, optimizer, device, model):
    _batch_losses = []
    
    _model = model
    for _ in trange(epochs):
        _model = _model.train()

        # Actual (batch-wise) training step
        for _, (_images, _labels) in enumerate(train_loader):
            _images = _images.to(device)
            _labels = _labels.to(device)

            _logits = _model(_images)
            _loss = criterion(_logits, _labels)
            _batch_losses.append(_loss.item())  # Store the loss for plotting, per batch

            optimizer.zero_grad()
            _loss.backward()
            optimizer.step()
    
    return _model

In [ ]:
## Define function used to get labels
def get_predicted_labels(model, test_data, device):
    test_data_tensor = th.tensor(test_data.reshape(-1, 1, 28, 28))

    model = model.eval()

    labels = []
    with th.no_grad():
        for i in range(test_data_tensor.shape[0]):
            data = test_data_tensor[i].reshape(1, 1, 28, 28)
            pred = model(data.to(device))
            labels.append(th.argmax(pred).item())
            
    return np.array(labels)

In [ ]:
# Define the Fully Connected Neural Network

class FullyConnectedNN_1layer(nn.Module):
    def __init__(self, image_dim, n_classes):
        
        super(FullyConnectedNN_1layer, self).__init__()
        
        self.fc1 = nn.Linear(in_features = image_dim,
                            out_features = n_classes)
    
    def forward(self, x):
        x = x.flatten(start_dim = 1)
        x = self.fc1(x)
        # x = F.relu(x)
        x = F.log_softmax(x, dim = 1)
        return x

In [ ]:
# Choose device
device = th.device("cuda" if th.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Train the model and calculate accuracy on the test set

if train_multiple_models:
    # Choose for which epochs to train the model
    epochs = np.arange(1, 21, 1)

    # Store the accuracies and predicted labels in two arrays
    trained_acc_FC_1l = []
    labels_FC_1l = np.ndarray((3000, len(epochs)))
    
    # Choose the loss  
    criterion = nn.CrossEntropyLoss()

    # Create a vectors to store the training time (column 1) for each epoch (column 0)
    times = np.ndarray((len(epochs), 2))

    for i in epochs:
        model = FullyConnectedNN_1layer(image_dim = 28 * 28, n_classes = 10).to(device)

        optimizer = th.optim.SGD(model.parameters(), lr=0.01)
        
        model = model.eval()

        untrained_loss, untrained_acc = get_test_stats(model, criterion, data_train_loader, device)


        model = model.train()
        
        # Keep track of how much time is required to train the model
        start_time = time.time()
        model = train_model(epochs = i,
                            train_loader = data_train_loader,
                            criterion = criterion,
                            optimizer = optimizer,
                            device = device,
                            model = model)
        end_time = time.time()
        
        times[i - 1, 0] = i
        times[i - 1, 1] = end_time - start_time
        
        model = model.eval()
                
        trained_loss, trained_acc = get_test_stats(model, criterion, data_test_loader, device)
        trained_acc_FC_1l.append(trained_acc)
        
        
        labels_FC_1l[:, i - 1] = get_predicted_labels(model = model, test_data = x_test_ex3, device = device)

        print(f"Epochs: {i} | Accuracy: untrained: {untrained_acc:.4f} | Trained: {trained_acc:.4f} | Time: {times[i - 1, 1]:.2f} s")


In [ ]:
# Plot the accuracy as a function of the number of epochs

if train_multiple_models:
    plt.plot(
        np.arange(1, len(trained_acc_FC_1l) + 1, 1),
        trained_acc_FC_1l,
        color=bar_rgb_color,
    )
    plt.xticks(np.arange(1, len(trained_acc_FC_1l) + 1, 2))
    plt.xlabel("Number of epochs")
    plt.ylabel("Accuracy", rotation=0, labelpad=20)
    plt.ylim(50, 100)
    plt.savefig("Report/ex3_FCNN1l_accuracy-epochs.png")

In [ ]:
# Try with two layers
class FullyConnectedNN_2layer(nn.Module):
    def __init__(self, image_dim, n_classes, hidden_features):
        
        super(FullyConnectedNN_2layer, self).__init__()
        
        self.fc1 = nn.Linear(in_features = image_dim,
                            out_features = hidden_features)
        
        self.fc2 = nn.Linear(in_features = hidden_features,
                             out_features = n_classes)
    
    def forward(self, x):
        x = x.flatten(start_dim = 1)
        x = self.fc1(x)
        # x = F.relu(x)
        x = self.fc2(x)
        # x = F.relu(x)
        x = F.log_softmax(x, dim = 1)
        return x

In [ ]:
# See how accuracy vary with the number of hidden neurons

if train_multiple_models:
    # Choose for which numbers of neurons to train the model
    neurons = np.arange(50, 10050, 100)

    trained_acc_FC_2l_neurons = []
    labels_FC_2l = np.ndarray((len(x_test_ex3), len(neurons)))

    times = np.ndarray((len(neurons), 2))

    # Choose the loss
    criterion = nn.CrossEntropyLoss()

    for i in range(len(neurons)):
        model = FullyConnectedNN_2layer(image_dim = 28 * 28, n_classes = 10, hidden_features = neurons[i])
        
        model = model.to(device)
        print(f"Using device: {device}")

        optimizer = th.optim.SGD(model.parameters(), lr = 0.01)
        model = model.eval()
        untrained_loss, untrained_acc = get_test_stats(model, criterion, data_train_loader, device)
        model = model.train()
        start_time = time.time()
        model = train_model(epochs = 8,
                            train_loader = data_train_loader,
                            criterion = criterion,
                            optimizer = optimizer,
                            device = device,
                            model = model)
        
        times[i, 0] = neurons[i]
        times[i, 1] = time.time() - start_time
        
        model = model.eval()

        trained_loss, trained_acc = get_test_stats(model, criterion, data_test_loader, device)
        trained_acc_FC_2l_neurons.append(trained_acc)
        
        print(f"Epochs: {i} | Accuracy: untrained: {untrained_acc:.4f} | Trained: {trained_acc:.4f} | Time: {times[i, 1]:.2f} s")
        
        model = model.eval()
        labels_FC_2l[:, i] = get_predicted_labels(test_data = x_test_ex3, device=device, model = model)

In [ ]:
# Plot accuracy wrt number of neurons

if train_multiple_models:
    plt.plot(
        np.arange(50, 50 + 100 * len(trained_acc_FC_2l_neurons), 100),
        trained_acc_FC_2l_neurons,
        color=bar_rgb_color,
    )
    plt.ylim((50, 100))
    plt.xlabel("Number of hidden neurons")
    plt.ylabel("Accuracy", rotation=0, labelpad=20)

    plt.savefig("Report/ex3_FCNN2l_accuracy-neurons.png")

In [ ]:
# Test how accuracy varies depending on the number of epochs

if train_multiple_models:
    # Define vector to keep all the accuracies, that we will plot
    trained_acc_FC_2l = []

    # Define an array to keep all the predicted labels
    labels_FC_2l = np.ndarray((len(x_test_ex3), len(epochs)))

    # Choose loss
    criterion = nn.CrossEntropyLoss()

    times = np.ndarray((len(epochs), 2))


    for i in range(len(epochs)):
        model = FullyConnectedNN_2layer(image_dim = 28 * 28, n_classes = 10, hidden_features = 850)
        
        model = model.to(device)
        print(f"Using device: {device}")

        optimizer = th.optim.SGD(model.parameters(), lr = 0.01)
        

        model = model.eval()

        untrained_loss, untrained_acc = get_test_stats(model, criterion, data_train_loader, device)

        print(f"Untrained test loss: {untrained_loss:.4f}, accuracy: {untrained_acc:.2f}%")
        
        model = model.train()
        
        start_time = time.time()
        model = train_model(epochs = epochs[i],
                            train_loader = data_train_loader,
                            criterion = criterion,
                            optimizer = optimizer,
                            device = device,
                            model = model)
        
        times[i, 0] = epochs[i]
        times[i, 1] = time.time() - start_time
        
        model = model.eval()

        trained_loss, trained_acc = get_test_stats(model, criterion, data_test_loader, device)
        trained_acc_FC_2l.append(trained_acc)
        
        print(f"Epochs: {i} | Accuracy: untrained: {untrained_acc:.4f} | Trained: {trained_acc:.4f} | Time: {times[i, 1]:.2f} s")
        
        model = model.eval()
        labels_FC_2l[:, i] = get_predicted_labels(test_data = x_test_ex3, device=device, model = model)

In [ ]:
# Plot the accuracy wrt the number of epochs used to train the model

if train_multiple_models:
    plt.plot(np.arange(1, 21, 1), trained_acc_FC_2l, color=bar_rgb_color)
    plt.xticks(np.arange(1, 21, 2))
    plt.ylim((50, 100))
    plt.xlabel("Number of epochs")
    plt.ylabel("Accuracy", rotation=0, labelpad=20)

    plt.savefig("Report/ex3_FCNN2l_accuracy-epochs.png")

In [ ]:
print(f"type of x_test = {x_test_ex3.dtype}")
print(f"type of x_test_ex3= {x_test_ex3.dtype}")

In [ ]:
# Choose one model and a set of parameter to predict the labels

times = []
fcnn_test_accuracies = []

for i in range(10):
    model = FullyConnectedNN_2layer(
        image_dim=28 * 28, n_classes=10, hidden_features=4000
    )

    model = model.to(device)

    optimizer = th.optim.SGD(model.parameters(), lr=0.01)

    criterion = nn.CrossEntropyLoss()

    model = model.eval()

    untrained_loss, untrained_acc = get_test_stats(
        model, criterion, data_train_loader, device
    )

    model = model.train()

    start_time = time.time()
    model = train_model(
        epochs=9,
        train_loader=data_train_loader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        model=model,
    )

    elapsed_time = time.time() - start_time

    model = model.eval()

    trained_loss, trained_acc = get_test_stats(
        model, criterion, data_test_loader, device
    )

    print(
        f"Accuracy: untrained: {untrained_acc:.4f} | Trained: {trained_acc:.4f} | Time: {elapsed_time:.2f} s"
    )
    times.append(elapsed_time)
    fcnn_test_accuracies.append(trained_acc)

model = model.eval()
labels_FCNN = get_predicted_labels(test_data=np.float32(x_test_ex3), device=device, model=model)

In [ ]:
print(
    f"Average time: {np.mean(times):.2f} s | Max time: {np.max(times):.2f} s | Min time: {np.min(times):.2f} s"
)
print(
    f"Average accuracy: {np.mean(fcnn_test_accuracies):.2f}% | Max accuracy: {np.max(fcnn_test_accuracies):.2f}% | Min accuracy: {np.min(fcnn_test_accuracies):.2f}%"
)

#### 3.3: Convolutional Neural Network

In [ ]:
class CNN_1layer(nn.Module):
    def __init__(self, n_classes, kernel_size, input_size):
        super(CNN_1layer, self).__init__()
        
        self._n_classes = n_classes
        
        self._padding = 0 # Default value
        
        self._stride = 1 # Default value
        
        self._stride_inv = 1 / self._stride
        
        self._kernel = kernel_size
        
        self._dimensions = input_size
        
 
        self.conv1 = nn.Conv2d(in_channels=self._dimensions[1],
                               out_channels = self._dimensions[1],
                               kernel_size=self._kernel)
        
        self.bn1 = nn.BatchNorm2d(self._dimensions[1])
        
        self._dimensions = [self._dimensions[0],
                            self._dimensions[1] ,
                            (self._dimensions[2] - self._kernel + 2 * self._padding) * self._stride_inv + 1,
                            (self._dimensions[3] - self._kernel + 2 * self._padding) * self._stride_inv + 1
                            ]
        
            
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = self._stride)
        
        self._dimensions = [self._dimensions[0],
                                self._dimensions[1] ,
                                (self._dimensions[2] - 2) * self._stride_inv + 1,
                                (self._dimensions[3] - 2) * self._stride_inv + 1
                                ]
        
        
        self.fc1 = nn.Linear(in_features = int(self._dimensions[2] * self._dimensions[3]), out_features = self._n_classes)
    
        
    def forward(self, x):
        x = self.conv1(x)
        
        x = self.bn1(x)
        
        x = F.relu(x)
        
        x = self.pool(x)

        x = x.view(x.shape[0], -1)
        
        x = self.fc1(x)
        
        x = F.log_softmax(x, dim = 1)
        
        return x

In [ ]:
# Train the model for different number of epochs

if train_multiple_models:
    model = CNN_1layer(n_classes = 10, kernel_size = 3, input_size=[7000, 1, 28, 28])

    # Choose loss
    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.0001)
    # Define vector to keep all the accuracies, that we will plot
    trained_acc_CNN_1l = []

    epochs = [1, 10, 20]

    CNN_labels = np.ndarray((len(x_test_ex3), len(epochs)))

    times = np.ndarray((len(epochs), 2))

    for i in range(len(epochs)):

        optimizer = th.optim.SGD(model.parameters(), lr = 0.01)


        model = model.to(device)

        model = model.eval()

        untrained_loss, untrained_acc = get_test_stats(model, criterion, train_loader, device)
        
        model = model.train()
        
        start_time = time.time()
        model = train_model(epochs = epochs[i],
                            train_loader = data_train_loader,
                            criterion = criterion,
                            optimizer = optimizer,
                            device = device,
                            model = model)
        
        times[i - 1, 0] = epochs[i]
        times[i - 1, 1] = time.time() - start_time
            
        model = model.eval()

        trained_loss, trained_acc = get_test_stats(model, criterion, data_test_loader, device)
        print(f"Epochs: {epochs[i]} | Accuracy: untrained: {untrained_acc:.4f} | Trained: {trained_acc:.4f} | Time: {times[i, 1]:.2f} s")
        
        trained_acc_CNN_1l.append(trained_acc)

        model = model.eval()

        CNN_labels[:, i] = get_predicted_labels(model = model, test_data = x_test_ex3, device = device)

In [ ]:
# Plot the accuracy wrt the number of epochs used to train the model
if train_multiple_models:
    plt.plot(epochs, trained_acc_CNN_1l, color=bar_rgb_color)
    plt.xticks(epochs)
    plt.xlabel("Number of epochs")
    plt.ylabel("Accuracy", rotation=0, labelpad=15)
    plt.ylim(50, 100)
    plt.savefig("Report/ex3_CNN1l_accuracy-epochs.png")

In [ ]:
# Try with two layers

class CNN_2layer(nn.Module):
    def __init__(self, n_classes, kernel_size, input_size, hidden_size):
        super(CNN_2layer, self).__init__()
        
        self._n_classes = n_classes
        
        self._padding = 0 # Default value
        
        self._stride = 1 # Default value
        
        self._stride_inv = 1 / self._stride
        
        self._kernel = kernel_size
        
        self._dimensions = input_size
        
 
        self.conv1 = nn.Conv2d(in_channels=self._dimensions[1],
                               out_channels = hidden_size,
                               kernel_size=self._kernel)
        
        self.bn1 = nn.BatchNorm2d(hidden_size)
        
        self._dimensions = [self._dimensions[0],
                            self._dimensions[1] ,
                            (self._dimensions[2] - self._kernel + 2 * self._padding) * self._stride_inv + 1,
                            (self._dimensions[3] - self._kernel + 2 * self._padding) * self._stride_inv + 1
                            ]
        
            
        self.pool1 = nn.MaxPool2d(2, stride = self._stride)
        
        self._dimensions = [self._dimensions[0],
                                self._dimensions[1] ,
                                (self._dimensions[2] - 2) * self._stride_inv + 1,
                                (self._dimensions[3] - 2) * self._stride_inv + 1
                                ]
        
        self.conv2 = nn.Conv2d(in_channels=hidden_size, out_channels=self._dimensions[1], kernel_size=self._kernel)
        
        self._dimensions = [self._dimensions[0],
                            self._dimensions[1] ,
                            (self._dimensions[2] - self._kernel + 2 * self._padding) * self._stride_inv + 1,
                            (self._dimensions[3] - self._kernel + 2 * self._padding) * self._stride_inv + 1
                            ]
        
        self.bn2 = nn.BatchNorm2d(self._dimensions[1])
        
        self.pool2 = nn.MaxPool2d(2, stride = self._stride)
        
        self._dimensions = [self._dimensions[0],
                                self._dimensions[1] ,
                                (self._dimensions[2] - 2) * self._stride_inv + 1,
                                (self._dimensions[3] - 2) * self._stride_inv + 1
                                ]
        
        self.fc1 = nn.Linear(in_features = int(self._dimensions[2] * self._dimensions[3]), out_features = self._n_classes)
    
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.log_softmax(x, dim = 1)

        return x

In [ ]:
# Train the model for different number of epochs`

if train_multiple_models:
    model = CNN_2layer(n_classes = 10, kernel_size = 3, input_size=[7000, 1, 28, 28], hidden_size=100)
    criterion = nn.CrossEntropyLoss()
    trained_acc_CNN_2l = []
    CNN_labels = np.ndarray((len(x_test_ex3), len(epochs)))
    times = np.ndarray((len(epochs), 2))

    for i in range(len(epochs)):

        optimizer = th.optim.SGD(model.parameters(), lr = 0.01)


        model = model.to(device)
        print(f"Using device: {device}")

        model = model.eval()

        untrained_loss, untrained_acc = get_test_stats(model, criterion, train_loader, device)

        model = model.train()
        
        start_time = time.time()
        model = train_model(epochs = epochs[i],
                            train_loader = data_train_loader,
                            criterion = criterion,
                            optimizer = optimizer,
                            device = device,
                            model = model)
        times[i, 0] = epochs[i]
        times[i, 1] = time.time() - start_time
        model = model.eval()

        trained_loss, trained_acc = get_test_stats(model, criterion, data_test_loader, device)
        print(f"Epochs: {epochs[i]} | Accuracy: untrained: {untrained_acc:.4f} | Trained: {trained_acc:.4f} | Time: {times[i, 1]:.2f} s")
        
        trained_acc_CNN_2l.append(trained_acc)

        model = model.eval()

        CNN_labels[:, i] = get_predicted_labels(model = model, test_data = x_test_ex3, device = device)

In [ ]:
# Plot the accuracy wrt the number of epochs used to train the model

if train_multiple_models:
    plt.plot(epochs, trained_acc_CNN_2l, color=bar_rgb_color)
    plt.xticks(epochs)
    plt.xlabel("Number of epochs")
    plt.ylabel("Accuracy", rotation=0, labelpad=20)
    plt.ylim(50, 100)
    plt.savefig("Report/ex3_CNN2l_accuracy-epochs.png")

In [ ]:
# Train the model for different number of neurons

if train_multiple_models:
    criterion = nn.CrossEntropyLoss()

    trained_acc_CNN_2l = []

    neurons = [50, 500, 1000]

    times = np.ndarray((len(neurons), 2))

    CNN_labels = np.ndarray((len(x_test), len(neurons)))

    for i in range(len(neurons)):
        model = CNN_2layer(n_classes = 10, kernel_size = 3, input_size=[7000, 1, 28, 28], hidden_size = neurons[i])
        
        optimizer = th.optim.SGD(model.parameters(), lr = 0.01)
        
        model = model.to(device)
        print(f"Using device: {device}")

        model = model.eval()

        untrained_loss, untrained_acc = get_test_stats(model, criterion, train_loader, device)

        print(f"Untrained test loss: {untrained_loss:.4f}, accuracy: {untrained_acc:.2f}%")

        model = model.train()
        start_time = time.time()
        model = train_model(epochs = 2,
                            train_loader = data_train_loader,
                            criterion = criterion,
                            optimizer = optimizer,
                            device = device,
                            model = model)
        times[i, 0] = neurons[i]
        times[i, 1] = time.time() - start_time
        model = model.eval()

        trained_loss, trained_acc = get_test_stats(model, criterion, data_test_loader, device)
        
        print(f"Epochs: {epochs[i]} | Accuracy: untrained: {untrained_acc:.4f} | Trained: {trained_acc:.4f} | Time: {times[i, 1]:.2f} s")
        
        trained_acc_CNN_2l.append(trained_acc)

        model = model.eval()

        CNN_labels[:, i] = get_predicted_labels(model = model, test_data = x_test_ex3, device = device)

In [ ]:
# Plot the accuracy wrt the number of neurons per hidden layer
if train_multiple_models:
    plt.plot(neurons, trained_acc_CNN_2l, color= bar_rgb_color)
    plt.xticks(neurons)
    plt.xlabel('Number of neurons per hidden layer')
    plt.ylabel('Accuracy', rotation=0, labelpad=20)
    plt.ylim(50, 100)
    plt.savefig("Report/ex3_CNN2l_accuracy-neurons.png")

In [ ]:
# Choose one model and a set of parameter to predict the labels
times = []
cnn_test_accuracies = []

for i in range(10):
    model = CNN_2layer(
        n_classes=10, kernel_size=3, input_size=[7000, 1, 28, 28], hidden_size=50
    )

    model = model.to(device)

    optimizer = th.optim.SGD(model.parameters(), lr=0.01)

    criterion = nn.CrossEntropyLoss()

    model = model.eval()

    untrained_loss, untrained_acc = get_test_stats(
        model, criterion, data_train_loader, device
    )

    model = model.train()

    start_time = time.time()
    model = train_model(
        epochs=5,
        train_loader=data_train_loader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        model=model,
    )

    elapsed_time = time.time() - start_time
    times.append(elapsed_time)

    model = model.eval()

    trained_loss, trained_acc = get_test_stats(
        model, criterion, data_test_loader, device
    )
    cnn_test_accuracies.append(trained_acc)

    print(
        f"Accuracy: untrained: {untrained_acc:.4f} | Trained: {trained_acc:.4f} | Time: {elapsed_time:.2f} s"
    )

model = model.eval()
labels_CNN = get_predicted_labels(test_data=np.float32(x_test_ex3), device=device, model=model)

In [ ]:
print(
    f"Average time: {np.mean(times):.2f} s | Max time: {np.max(times):.2f} s | Min time: {np.min(times):.2f} s"
)
print(
    f"Average accuracy: {np.mean(cnn_test_accuracies):.2f}% | Max accuracy: {np.max(cnn_test_accuracies):.2f}% | Min accuracy: {np.min(cnn_test_accuracies):.2f}%"
)

In [ ]:
# Define a 2-layer Fully Convolutional Network

class FullyConv_2layer(nn.Module):
    def __init__(self, n_classes, kernel_size, input_size, hidden_size):
        super(FullyConv_2layer, self).__init__()
        
        self._n_classes = n_classes
        
        self._padding = 0 # Default value
        
        self._stride = 1 # Default value
        
        self._stride_inv = 1 / self._stride
        
        self._kernel = kernel_size
        
        self._dimensions = input_size
        
 
        self.conv1 = nn.Conv2d(in_channels=self._dimensions[1],
                               out_channels = hidden_size,
                               kernel_size=self._kernel)
        
        self.bn1 = nn.BatchNorm2d(hidden_size)
        
        self._dimensions = [self._dimensions[0],
                            self._dimensions[1] ,
                            (self._dimensions[2] - self._kernel + 2 * self._padding) * self._stride_inv + 1,
                            (self._dimensions[3] - self._kernel + 2 * self._padding) * self._stride_inv + 1
                            ]
        
            
        self.pool1 = nn.MaxPool2d(2, stride = self._stride)
        
        self._dimensions = [self._dimensions[0],
                                self._dimensions[1] ,
                                (self._dimensions[2] - 2) * self._stride_inv + 1,
                                (self._dimensions[3] - 2) * self._stride_inv + 1
                                ]
        
        self.conv2 = nn.Conv2d(in_channels=hidden_size, out_channels=self._dimensions[1], kernel_size=self._kernel)
        
        self._dimensions = [self._dimensions[0],
                            self._dimensions[1] ,
                            (self._dimensions[2] - self._kernel + 2 * self._padding) * self._stride_inv + 1,
                            (self._dimensions[3] - self._kernel + 2 * self._padding) * self._stride_inv + 1
                            ]
        
        self.bn2 = nn.BatchNorm2d(self._dimensions[1])
        
        self.pool2 = nn.MaxPool2d(2, stride = self._stride)
        
        self._dimensions = [self._dimensions[0],
                                self._dimensions[1] ,
                                (self._dimensions[2] - 2) * self._stride_inv + 1,
                                (self._dimensions[3] - 2) * self._stride_inv + 1
                                ]
    
        
    def forward(self, x):
        
        x = self.conv1(x)
        
        x = self.bn1(x)
        
        x = F.relu(x)
        
        x = self.pool1(x)
        
        x = self.conv2(x)
        
        x = self.bn2(x)
        
        x = F.relu(x)
        
        x = self.pool2(x)
        
        x = x.view(x.shape[0], -1)

        x = F.log_softmax(x, dim = 1)

        return x

In [ ]:
model = FullyConv_2layer(n_classes = 10, kernel_size = 3, input_size=[7000, 1, 28, 28], hidden_size = 250)

optimizer = th.optim.SGD(model.parameters(), lr = 0.01)


model = model.to(device)
print(f"Using device: {device}")

model = model.eval()

untrained_loss, untrained_acc = get_test_stats(model, criterion, train_loader, device)

print(f"Untrained test loss: {untrained_loss:.4f}, accuracy: {untrained_acc:.2f}%")

model = model.train()
start_time = time.time()
model = train_model(epochs = 2,
                    train_loader = data_train_loader,
                    criterion = criterion,
                    optimizer = optimizer,
                    device = device,
                    model = model)
elapsed_time = time.time() - start_time
model = model.eval()

trained_loss, trained_acc = get_test_stats(model, criterion, data_test_loader, device)
print(f"Accuracy: untrained: {untrained_acc:.4f} | Trained: {trained_acc:.4f} | Time: {elapsed_time:.2f} s")

### Section 4: Wrap-up!

Evaluate the overall accuracy of the pipeline on the *test set* of *FashionMNIST*. *I.e.* compare the predicted labels from the three classifiers built in *Section 3* with the true labels.

In order to assign a true label *name* (e.g. *trousers*, *sandal*, ...) to those determined just from *(kernel-)PCA* (that obviously carry no direct information about the subject of the picture), you can either:

i. *Cheat* and use the most abundant labels for each group of *(kernel-)PCA-labelled* datapoints.

ii. Sample a subset of datapoints from each *(kernel-)PCA-labelled* class, and assign one label by direct visual inspection. If you choose this route, it may also serve as a reminder of the fact that *expert labelling* is not always a trivial (and almost never a fast) task!

Comment on the results obtained.

In [ ]:
print(train_subset_scaled.shape)
print(labels_Spectral.shape)
print(data_pca_sigmoid.shape)
print(label_predict_SVC_linear.shape)
print(label_predict_SVC_sigmoid.shape)
print(label_predict_SVC_rbf.shape)
print(labels_FCNN.shape)
print(labels_CNN.shape)

Creating training and testing datasets. Also printing an image and its label. This is to remind us that the labels are given by the Spectral clustering and therefore can be wrong. 

In [ ]:
# Here the labels are taken from the Spectral clustering!!
idx = 16

plt.figure(figsize=(2, 2))
plt.imshow(x_test[idx].reshape(28, 28), cmap="gray")

##plt.show()

print(description[labels_Spectral[idx].item()])
print(x_train.shape)
print(train_subset_scaled.shape)

In [ ]:
def print_img_index(indices, img_set, lab_set, class_number):
    #print_img_index(indices, x_test_ex3, true_labels, class_number)
    counter = np.zeros(10)

    # Counting number of appearance of each label
    for i in range(len(indices)):
        label = lab_set[indices[i]]
        counter[label] += 1

    # Plotting the images with the labels as titles
    fig, axs = plt.subplots(5, 6, figsize=(10, 10))
    axs = axs.flatten()

    for i, idx in enumerate(indices):
        axs[i].imshow(
            img_set[idx].reshape(28, 28), cmap="gray"
        )  # Assuming images are 28x28 pixels
        axs[i].axis("off")  # Turn off axis labels for cleaner visualization
        # string = str(description[labels_subset[idx]])
        axs[i].set_title(description[lab_set[idx].item()])
        #axs[i].set_title(lab_set[idx].item())

    fig.suptitle(f"Class {class_number}")

    # Printing the counted amounts
    for j in range(10):
        print(f"{description[j]}: {counter[j]}")
    
    return np.argmax(counter)

In [ ]:
# Gets 30 samples for the specified class according to given labels
def show_class(class_number, learnt_labels, true_labels):
    # show_class(i, labels_CNN, y_test)
    indices = []
    counter = np.zeros(10)
    idx = np.random.choice(len(learnt_labels), size=1, replace=False)[0]

    # Selecting 30 samples with learnt_label = class number
    for i in range(30):
        idx = np.random.choice(len(learnt_labels), size=1, replace=False)[0]
        while learnt_labels[idx] != class_number:
            idx = np.random.choice(len(learnt_labels), size=1, replace=False)[0]

        indices.append(idx)
        counter[true_labels[idx]] += 1

    # Now that the indices are selected, print the images and the corresponding label on which the model was trained
    physical_class = print_img_index(
        indices=indices,
        img_set=x_test_ex3,
        lab_set=true_labels,
        class_number=class_number,
    )
    return physical_class.astype(int)

In [ ]:
physical_classes = np.zeros(10)
for i in range(10):
    print(f"\nClass {i}:")
    physical_classes[i] = show_class(
        class_number=i, learnt_labels=labels_CNN, true_labels=y_test
    )

physical_classes = physical_classes.astype(int)

In [ ]:
# Creating a mapping from one set of 10 integer numbers to another set of 10 integer numbers
pred_class = [i for i in range(10)]
class_mapping = dict(zip(pred_class, physical_classes))

# Example usage:
input_value = 4
output_value = class_mapping.get(input_value)

if output_value is not None:
    print(f"The mapped value for {input_value} is {description[output_value]}")
else:
    print(f"No mapping found for {input_value}")

In [ ]:
print(physical_classes)

In [ ]:
predicted_labels = np.array([])
for element in labels_CNN:
    predicted_labels = np.append(predicted_labels, class_mapping.get(element))
overall_accuracy_CNN = accuracy_score(y_true=y_test, y_pred=predicted_labels)
print(f"CNN overall accuracy: {overall_accuracy_CNN:.2f}")
del predicted_labels

predicted_labels = np.array([])
for element in labels_FCNN:
    predicted_labels = np.append(predicted_labels, class_mapping.get(element))
overall_accuracy_FCNN = accuracy_score(y_true=y_test, y_pred=predicted_labels)
print(f"FCNN overall accuracy: {overall_accuracy_FCNN:.2f}")
del predicted_labels

predicted_labels = np.array([])
for element in label_predict_SVC_linear:
    predicted_labels = np.append(predicted_labels, class_mapping.get(element))
overall_accuracy_linear = accuracy_score(y_true=y_test, y_pred=predicted_labels)
print(f"Linear SVM overall accuracy: {overall_accuracy_linear:.2f}")

predicted_labels = np.array([])
for element in label_predict_SVC_rbf:
    predicted_labels = np.append(predicted_labels, class_mapping.get(element))
overall_accuracy_rbf = accuracy_score(y_true=y_test, y_pred=predicted_labels)
print(f"RBF SVM overall accuracy: {overall_accuracy_rbf:.2f}")

predicted_labels = np.array([])
for element in label_predict_SVC_poly:
    predicted_labels = np.append(predicted_labels, class_mapping.get(element))
overall_accuracy_poly = accuracy_score(y_true=y_test, y_pred=predicted_labels)
print(f"Polynomial SVM overall accuracy: {overall_accuracy_poly:.2f}")

predicted_labels = np.array([])
for element in label_predict_SVC_sigmoid:
    predicted_labels = np.append(predicted_labels, class_mapping.get(element))
overall_accuracy_sigmoid = accuracy_score(y_true=y_test, y_pred=predicted_labels)
print(f"Sigmoid SVM overall accuracy: {overall_accuracy_sigmoid:.2f}")

In [ ]:
# Horizontal bar chart
import pandas as pd

methods = ["SVC linear", "SVC RBF", "SVC Polynomial", "SVC sigmoid", "FCNN", "CNN"]
accuracies = np.round(
    [
        100 * overall_accuracy_linear,
        100 * overall_accuracy_rbf,
        100 * overall_accuracy_poly,
        100 * overall_accuracy_sigmoid,
        100 * overall_accuracy_FCNN,
        100 * overall_accuracy_CNN
    ],
    2,
)
print(accuracies)
df = pd.DataFrame({"methods": methods, "accuracies": accuracies})
df.sort_values("accuracies", inplace=True, ascending=False)

print(df)

fig, ax = plt.subplots()
bars = ax.barh(y=df.methods, width=df.accuracies, color = bar_rgb_color)
ax.set_yticks(np.arange(len(df.methods)))
ax.set_xlabel("Accuracy (%)")
ax.invert_yaxis()
for bar in bars:
    plt.text(
        bar.get_width(),
        bar.get_y() + bar.get_height() / 2,
        f"{bar.get_width():.2f}",
        va="center",
        ha="right",
        fontsize=8,
        color="white",
    )

##plt.show()

plt.savefig("Report/ex3_overall_accuracies.png")

### Section 5: A *fully-supervised* approach

Repeat the steps of *Section 3* using the true labels of the dataset. Comment on the results, and draw a comparison between such results and those obtained from the previous *hybrid* pipeline.

In [ ]:
# Split train and test dataset as before, but this time use the true labels

x_train, x_test, y_train, y_test = train_test_split(train_subset_scaled, labels, test_size = 0.3, random_state = 42, shuffle= False)

In [ ]:
classifier = SVC(kernel = "linear").fit(x_train, y_train.reshape(len(y_train)))

label_predict_SVC_linear = classifier.predict(x_test)

In [ ]:
classifier = SVC(kernel = "rbf").fit(x_train, y_train.reshape(len(y_train)))

label_predict_SVC_rbf = classifier.predict(x_test)

In [ ]:
classifier = SVC(kernel = "poly").fit(x_train, y_train.reshape(len(y_train)))

label_predict_SVC_poly = classifier.predict(x_test)

In [ ]:
classifier = SVC(kernel = "sigmoid").fit(x_train, y_train.reshape(len(y_train)))

label_predict_SVC_sigmoid = classifier.predict(x_test)

In [ ]:
print(f"Accuracy: Linear {accuracy_score(y_test, label_predict_SVC_linear):.2} | RBF {accuracy_score(y_test, label_predict_SVC_rbf):.2} | Poly {accuracy_score(y_test, label_predict_SVC_poly):.2} |Sigmoid {accuracy_score(y_test, label_predict_SVC_sigmoid):.2}")

In [ ]:
labels_SVC_true = np.concatenate((y_test.reshape(len(y_test), 1),
                             label_predict_SVC_linear.reshape(len(label_predict_SVC_linear), 1),
                             label_predict_SVC_rbf.reshape(len(label_predict_SVC_rbf), 1),
                             label_predict_SVC_poly.reshape(len(label_predict_SVC_poly), 1),
                             label_predict_SVC_sigmoid.reshape(len(label_predict_SVC_sigmoid), 1)), axis = 1)

In [ ]:
model = KernelPCA(n_components=2, kernel="sigmoid").fit_transform(x_test)

In [ ]:
titles = ["True labels", "Linear", "RBF", "Polynomial", "Sigmoid"]


fig = plt.figure(figsize=(9, 6), dpi=200)
gs = gridspec.GridSpec(2, 6)  # , width_ratios=[1, 1, 1], height_ratios=[1, 1])

# Create subplots
ax1 = plt.subplot(gs[0, :2])
ax2 = plt.subplot(gs[0, 2:4])
ax3 = plt.subplot(gs[0, 4:])
ax4 = plt.subplot(gs[1, 1:3])
ax5 = plt.subplot(gs[1, 3:5])

# Plot data in subplots (replace with your actual plotting code)
ax1.scatter(model[:, 0], model[:, 1], c=labels_SVC_true[:, 0], cmap=cmap, marker=".")
ax1.title.set_text(titles[0])

ax2.scatter(model[:, 0], model[:, 1], c=labels_SVC_true[:, 1], cmap=cmap, marker=".")
ax2.title.set_text(titles[1])

ax3.scatter(model[:, 0], model[:, 1], c=labels_SVC_true[:, 2], cmap=cmap, marker=".")
ax3.title.set_text(titles[2])

ax4.scatter(model[:, 0], model[:, 1], c=labels_SVC_true[:, 3], cmap=cmap, marker=".")
ax4.title.set_text(titles[3])

ax5.scatter(model[:, 0], model[:, 1], c=labels_SVC_true[:, 4], cmap=cmap, marker=".")
ax5.title.set_text(titles[4])

for ax in [ax1, ax2, ax3, ax4, ax5]:
    ax.set_xticks([])
    ax.set_yticks([])

# Adjust layout for better spacing
plt.tight_layout()

# Show the plot
##plt.show()

plt.savefig("Report/unsupervised_SVC_true_labels.png")

In [ ]:
#model = KernelPCA(n_components = 2, kernel = "sigmoid").fit_transform(x_test)

fig, axs = plt.subplots(1, 5, figsize = (15, 3), sharex=True, sharey=True)

titles = ["True labels", "Linear kernel", "RBF kernel", "Polynomial kernel", "Sigmoid kernel"]

i = 0
for ax in axs:
    ax.scatter(model[:, 0], model[:, 1], c = labels_SVC_true[:, i], cmap = cmap)
    ax.title.set_text(titles[i])
    i += 1



In [ ]:
labels = np.hstack((labels_SVC_unsupervised, labels_SVC_true))

In [ ]:
i = 0
title = ["Unsupervised"]
fig, axs = plt.subplots(3, 3, figsize = (9, 9), sharex=True, sharey=True)
for ax in axs:
    if i == 0 or i == 5:
        continue
    else:
        ax.scatter(model[:, 0], model[:, 1], c = labels[:, i], cmap = cmap)
        ax.title.set_text(titles[i])
        i += 1

#### 5.2: Fully Connected NN
Trying different numbers of layers and hidden features

In [ ]:
data_train = TensorDataset(Tensor(x_train.reshape(-1, 1, 28, 28)), th.tensor(y_train.reshape(len(y_train)), dtype = th.long))
data_train_loader = DataLoader(dataset = data_train, batch_size = BATCH_SIZE, shuffle = False)

data_test = TensorDataset(Tensor(x_test.reshape(-1, 1, 28, 28)), th.tensor(y_test.reshape(len(y_test)), dtype = th.long))
data_test_loader = DataLoader(dataset = data_test, batch_size = BATCH_SIZE, shuffle = False)

In [ ]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")

### 5.2
Fully Connected Neural Network

In [ ]:
# Train the Fully Connected Neural Network with 1 layer and calculate accuracy on the test set for different number of epochs

if train_multiple_models:
    # Choose the loss
    criterion = nn.CrossEntropyLoss()

    # optimizer = optim.Adam(model.parameters(), lr = 0.0001, weight_decay = 0.0001)

    model = model.eval()

    model = FullyConnectedNN_1layer(image_dim = 28 * 28, n_classes = 10).to(device)


    untrained_loss, untrained_acc = get_test_stats(model, criterion, train_loader, device)
    print(f"Untrained test loss: {untrained_loss:.4f}, accuracy: {untrained_acc:.2f}%")

    epochs = np.arange(1, 21, 1)

    trained_acc_FC_1l = []
    labels_FC_1l = np.ndarray((len(x_test), len(epochs)))

    for i in range(len(epochs)):
        
        model = FullyConnectedNN_1layer(image_dim = 28 * 28, n_classes = 10).to(device)

        optimizer = th.optim.SGD(model.parameters(), lr = 0.01)
        
        model = model.train()

        model = train_model(epochs = epochs[i],
                                train_loader = data_train_loader,
                                criterion = criterion,
                                optimizer = optimizer,
                                device = device,
                                model = model)
        model = model.eval()
                
        trained_loss, trained_acc = get_test_stats(model, criterion, data_test_loader, device)
        
        trained_acc_FC_1l.append(trained_acc)

        print(f"Accuracy: {trained_acc} | Loss: {trained_loss}")
        
        labels_FC_1l[:, i] = get_predicted_labels(test_data = x_test, device = device, model = model)

In [ ]:
# Plot the accuracy wrt the number of epochs used to train the model

if train_multiple_models:
    plt.plot(epochs, trained_acc_FC_1l, color=bar_rgb_color)
    plt.xticks(epochs[::2])
    plt.xlabel("Number of epochs")
    plt.ylabel("Accuracy", rotation=0, labelpad=20)
    plt.ylim(50, 100)
    plt.savefig("Report/ex5_FCNN1l_accuracy-epochs.png")

In [ ]:
# Train the Fully Connected Neural Network with 2 layers and calculate accuracy on the test set for different number of epochs

if train_multiple_models:
    # Choose the loss
    criterion = nn.CrossEntropyLoss()

    # optimizer = optim.Adam(model.parameters(), lr = 0.0001, weight_decay = 0.0001)

    model = model.eval()

    model = FullyConnectedNN_2layer(image_dim = 28 * 28, n_classes = 10, hidden_features=250).to(device)


    untrained_loss, untrained_acc = get_test_stats(model, criterion, train_loader, device)
    print(f"Untrained test loss: {untrained_loss:.4f}, accuracy: {untrained_acc:.2f}%")

    trained_acc_FC_2l = []
    labels_FC_2l = np.ndarray((len(x_test), len(epochs)))

    for i in range(len(epochs)):
        
        model = FullyConnectedNN_2layer(image_dim = 28 * 28, n_classes = 10, hidden_features=250).to(device)

        optimizer = th.optim.SGD(model.parameters(), lr = 0.01)
        
        model = model.train()

        model = train_model(epochs = epochs[i],
                                train_loader = data_train_loader,
                                criterion = criterion,
                                optimizer = optimizer,
                                device = device,
                                model = model)
        model = model.eval()
                
        trained_loss, trained_acc = get_test_stats(model, criterion, data_test_loader, device)
        
        trained_acc_FC_2l.append(trained_acc)

        print(f"Accuracy: {trained_acc} | Loss: {trained_loss}")
        
        labels_FC_2l[:, i] = get_predicted_labels(test_data = x_test, device = device, model = model)

In [ ]:
# Plot the accuracy wrt the number of epochs used to train the model

if train_multiple_models:
    plt.plot(epochs, trained_acc_FC_2l, color=bar_rgb_color)
    plt.xticks(epochs[::2])
    plt.xlabel("Number of epochs")
    plt.ylabel("Accuracy", rotation=0, labelpad=20)
    plt.ylim(50, 100)
    plt.savefig("Report/ex5_FCNN2l_accuracy-epochs.png")

In [ ]:
# Train the Convolutional Neural Network with 2 layers and calculate accuracy on the test set for different number of neurons

if train_multiple_models:
    # Choose the loss
    criterion = nn.CrossEntropyLoss()

    neurons = np.arange(50, 10050, 100)

    labels_FC_2l = np.ndarray((len(x_test), len(neurons)))
    trained_acc_FC_2l = []


    for i in range(len(neurons)):
        model = FullyConnectedNN_2layer(image_dim = 28 * 28, n_classes = 10, hidden_features = neurons[i]).to(device)
        optimizer = th.optim.SGD(model.parameters(), lr = 0.01)

        untrained_loss, untrained_acc = get_test_stats(model, criterion, train_loader, device)
        print(f"Untrained test loss: {untrained_loss:.4f}, accuracy: {untrained_acc:.2f}%")
        
        model = model.train()

        model = train_model(epochs = 2,
                                train_loader = data_train_loader,
                                criterion = criterion,
                                optimizer = optimizer,
                                device = device,
                                model = model)
        model = model.eval()
                
        trained_loss, trained_acc = get_test_stats(model, criterion, data_test_loader, device)
        
        trained_acc_FC_2l.append(trained_acc)

        print(f"Accuracy: {trained_acc} | Loss: {trained_loss}")
        
        labels_FC_2l[:, i] = get_predicted_labels(test_data = x_test, device = device, model = model)

    # print(f"Epochs: {epochs[i]} | Accuracy: untrained: {untrained_acc:.4f} | Trained: {trained_acc:.4f} | Time: {times[i, 1]:.2f} s")

In [ ]:
# Plot the accuracy wrt the number of neurons

if train_multiple_models:
    plt.plot(neurons, trained_acc_FC_2l, color=bar_rgb_color)
    plt.xlabel("Number neurons per hidden layer")
    plt.ylabel("Trained accuracy")
    plt.ylim(50, 100, rotation=0, labelpad=20)
    plt.savefig("Report/ex5_FCNN2l_accuracy-neurons.png")

In [ ]:
# Choose one model and a set of parameter to predict the labels

# Choose the loss
criterion = nn.CrossEntropyLoss()

model = FullyConnectedNN_2layer(image_dim = 28 * 28, n_classes = 10, hidden_features = 6000).to(device)
optimizer = th.optim.SGD(model.parameters(), lr = 0.01)

untrained_loss, untrained_acc = get_test_stats(model, criterion, train_loader, device)
print(f"Untrained test loss: {untrained_loss:.4f}, accuracy: {untrained_acc:.2f}%")

model = model.train()

model = train_model(epochs = 10,
                        train_loader = data_train_loader,
                        criterion = criterion,
                        optimizer = optimizer,
                        device = device,
                        model = model)
model = model.eval()
        
trained_loss, trained_acc = get_test_stats(model, criterion, data_test_loader, device)

print(f"Accuracy: {trained_acc} | Loss: {trained_loss}")

labels_FC_2l = get_predicted_labels(test_data = np.float32(x_test), device = device, model = model)

#### 5.3:
Convolutional and Fully Convolutional Neural Network

In [ ]:
train_multiple_models = True

In [ ]:
# Train the Convolutional Neural Network with 1 layer and calculate accuracy on the test set for different number of epochs

if train_multiple_models:
    # Device selection
    
    epochs = np.arange(1, 21, 1)
    
    trained_acc_CNN_1l = []
    labels_CNN_1l = np.ndarray((len(x_test), len(epochs)))
    
    for i in range(len(epochs)):
        model = CNN_1layer(n_classes = 10, kernel_size = 3, input_size = [7000, 1, 28, 28])

        criterion = nn.CrossEntropyLoss()

        # optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.0001)
        optimizer = optim.SGD(model.parameters(), lr = 0.01)

        

        model = model.to(device)

        model = model.eval()

        untrained_loss, untrained_acc = get_test_stats(model, criterion, train_loader, device)
        
        print(f"Untrained test loss: {untrained_loss:.4f}, accuracy: {untrained_acc:.2f}%")

        model = model.train()

        model = train_model(epochs = epochs[i],
                                    train_loader = data_train_loader,
                                    criterion = criterion,
                                    optimizer = optimizer,
                                    device = device,
                                    model = model)
        model = model.eval()
        trained_loss, trained_acc = get_test_stats(model, criterion, data_test_loader, device)
        trained_acc_CNN_1l.append(trained_acc)
        
        print(f"Accuracy: {trained_acc} | Loss: {trained_loss}")

        labels_CNN_1l[:, i] = get_predicted_labels(test_data= np.float32(x_test), model = model, device = device)

In [ ]:
# Plot the accuracy wrt the number of epochs used to train the model
if train_multiple_models:
    plt.plot(epochs, np.array(trained_acc_CNN_1l), color=bar_rgb_color)
    plt.xticks(epochs[::2])
    plt.xlabel("Number of epochs")
    plt.ylabel("Accuracy", rotation=0, labelpad=20)
    plt.ylim(50, 100)
    plt.savefig("Report/ex5_CNN1l_accuracy-epochs.png")

In [ ]:
# Train the Convolutional Neural Network with 2 layers and calculate accuracy on the test set for different number of neurons per hidden layer
if train_multiple_models:
    
    neurons = np.arange(50, 550, 100)
    
    labels_CNN_2l = np.ndarray((len(x_test), len(neurons)))
    
    trained_acc_CNN_2l = []
    
    for i in range(len(neurons)):
        model = CNN_2layer(n_classes = 10, kernel_size = 2, input_size = [7000, 1, 28, 28], hidden_size = neurons[i])

        criterion = nn.CrossEntropyLoss()

        optimizer = th.optim.SGD(model.parameters(), lr=0.01)

        model = model.to(device)

        model = model.eval()

        untrained_loss, untrained_acc = get_test_stats(model, criterion, data_train_loader, device)


        model = model.train()

        start_time = time.time()
        model = train_model(epochs = 3,
                            train_loader = data_train_loader,
                            criterion = criterion,
                            optimizer = optimizer,
                            device = device,
                            model = model)

        model = model.eval()
        times = time.time() - start_time
        trained_loss, trained_acc = get_test_stats(model, criterion, data_test_loader, device)
        print(f"Neurons: {neurons[i]} | Accuracy: untrained: {untrained_acc:.4f} | Trained: {trained_acc:.4f} | Time: {times:.2f} s")
        
        trained_acc_CNN_2l.append(trained_acc)

        model = model.eval()
        labels_CNN_2l[:, i] = get_predicted_labels(test_data = np.float32(x_test), model = model, device = device)

In [ ]:
# Plot the accuracy wrt the number of neurons per hidden layer

if train_multiple_models:
    plt.plot(neurons, trained_acc_CNN_2l, color=bar_rgb_color)
    plt.xlabel("Number neurons per hidden layer")
    plt.ylabel("Trained accuracy", rotation=0, labelpad=20)
    plt.ylim(50, 100)
    plt.savefig("Report/ex5_CNN2l_accuracy-neurons.png")

In [ ]:
# Choose one set of parameters

model = CNN_2layer(n_classes = 10, kernel_size = 2, input_size = [7000, 1, 28, 28], hidden_size = 200)

criterion = nn.CrossEntropyLoss()

# optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.0001)

optimizer = th.optim.SGD(model.parameters(), lr=0.01, momentum=0)


model = model.to(device)

model = model.eval()

untrained_loss, untrained_acc = get_test_stats(model, criterion, train_loader, device)

print(f"Untrained test loss: {untrained_loss:.4f}, accuracy: {untrained_acc:.2f}%")

model = model.train()

model = train_model(epochs = 3,
                    train_loader = data_train_loader,
                    criterion = criterion,
                    optimizer = optimizer,
                    device = device,
                    model = model)

model = model.eval()
trained_loss, trained_acc = get_test_stats(model, criterion, data_test_loader, device)
print(f"Accuracy: {trained_acc} | Loss: {trained_loss}")

model = model.eval()
labels_CNN_2l = get_predicted_labels(test_data = np.float32(x_test), model = model, device = device)